In [1]:
from importlib import import_module

In [2]:
from snntoolbox.parsing.utils import *

Using TensorFlow backend.
/home/wolin/.conda/envs/tf2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wolin/.conda/envs/tf2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wolin/.conda/envs/tf2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wolin/.conda/envs/tf2/lib/python3.7

In [3]:
from snntoolbox.parsing.model_libs.keras_input_lib import *

In [4]:
from snntoolbox.bin.utils import update_setup

In [5]:
from os import *

In [6]:
model_path = "/home/wolin/snn_toolbox/examples/models/lenet5/keras/"

In [7]:
config = update_setup(model_path + "config")







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

SNN toolbox Warning: Currently, turning off Poisson input is
only possible in INI simulator. Falling back on Poisson input.


/home/wolin/.conda/envs/tf2/lib/python3.7/site-packages/keras/utils/conv_utils.py:93: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])


Load the network from file and parse it

In [8]:
model = "98.96"

In [9]:
config

In [10]:
n = load(model_path, model)

/home/wolin/.conda/envs/tf2/lib/python3.7/site-packages/keras/utils/conv_utils.py:93: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])


In [11]:
mp = ModelParser(n['model'], config)

In [12]:
mp.parse()

Parsing layer Conv2D.
Using activation relu.
Parsing layer MaxPooling2D.
Parsing layer Conv2D.
Using activation relu.
Parsing layer MaxPooling2D.
Skipping layer Dropout.
Parsing layer Conv2D.
Using activation relu.
Parsing layer Flatten.
Parsing layer Dense.
Using activation relu.
Parsing layer Dense.
Using activation softmax.



In [13]:
pm = mp.build_parsed_model()

Building parsed model...

Compiling parsed model...



In [14]:
pm

In [15]:
from snntoolbox.conversion.utils import normalize_parameters

In [16]:
from snntoolbox.datasets.utils import get_dataset

Load the data and use it to normalize the activations

In [18]:
normset, testset = get_dataset(config)

Loading data set from '.npz' files in /home/wolin/snn_toolbox/examples/datasets/mnist.



In [45]:
print(keras.__version__)

2.1.6


In [38]:
normset['x_norm'].shape

(10000, 1, 28, 28)

In [38]:
normset['x_norm'].shape

(10000, 1, 28, 28)

In [19]:
normalize_parameters(pm, config, **normset)

Normalizing parameters...
Using 10000 samples for normalization.
INFO: Need ['0.14', '0.04', '0.08', '0.00', '0.00'] GB for layer activations.
May have to reduce size of data set used for normalization.
Loading activations stored during a previous run.
Scale factor: 1.03.
Loading activations stored during a previous run.
Scale factor: 1.48.
Loading activations stored during a previous run.
Scale factor: 2.44.
Loading activations stored during a previous run.
Scale factor: 10.97.
Loading activations stored during a previous run.
Scale factor: 1.00.
Using scale factor 1.00 for softmax layer.
Plotting distributions of weights and activations before and after normalizing...
Loading activations stored during a previous run.
Loading activations stored during a previous run.
Loading activations stored during a previous run.
Loading activations stored during a previous run.
Loading activations stored during a previous run.



In [20]:
pm

In [21]:
from snntoolbox.simulation.utils import *

In [22]:
from snntoolbox.simulation.target_simulators.brian2_target_sim import *

In [23]:
def import_target_sim(config):

    sim_str = config.get('simulation', 'simulator')
    code_str = '_' + config.get('conversion', 'spike_code') \
        if sim_str == 'INI' else ''

    return import_module('snntoolbox.simulation.target_simulators.'
                         + sim_str + code_str + '_target_sim')

In [24]:
target_sim = import_target_sim(config)

In [25]:
target_sim

<module 'snntoolbox.simulation.target_simulators.brian2_target_sim' from '/home/wolin/snn_toolbox/snntoolbox/simulation/target_simulators/brian2_target_sim.py'>

In [26]:
spiking_model = target_sim.SNN(config, None)

Initializing brian2 simulator...



In [27]:
spiking_model.build(pm)

WARNING    /home/wolin/.conda/envs/tf2/lib/python3.7/site-packages/keras/utils/conv_utils.py:93: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])
 [py.warnings]


Building spiking model...
Building layer: 00Conv2D_6x24x24
......

WARNING    /home/wolin/snn_toolbox/snntoolbox/simulation/target_simulators/brian2_target_sim.py:347: RuntimeWarning: Biases not implemented.
  warnings.warn("Biases not implemented.", RuntimeWarning)
 [py.warnings]



Connecting layer...


WARNING    /home/wolin/snn_toolbox/snntoolbox/simulation/utils.py:1241: RuntimeWarning: Layer type 'MaxPooling' not supported yet. Falling back on 'AveragePooling'.
  "Falling back on 'AveragePooling'.", RuntimeWarning)
 [py.warnings]


Building layer: 01MaxPooling2D_6x12x12
......
Building layer: 02Conv2D_16x8x8
................
Connecting layer...
Building layer: 03MaxPooling2D_16x4x4
................
Building layer: 04Conv2D_120x4x4
........................................................................................................................
Connecting layer...
Building layer: 05Flatten_1920
Building layer: 06Dense_84
Lenght of weights:161280
Building layer: 07Dense_10
Lenght of weights:840
Compiling spiking model...

Detected layer with biases: 00Conv2D_6x24x24
Detected layer with biases: 02Conv2D_16x8x8
Detected layer with biases: 04Conv2D_120x4x4
Detected layer with biases: 06Dense_84
Detected layer with biases: 07Dense_10
Number of operations of ANN: 2346734
Number of neurons: 7614
Number of synapses: 1397800



In [28]:
def run(snn, **test_set):
    return snn.run(**test_set)

In [29]:
spiking_model._duration

100

In [30]:
spiking_model.save(model_path, model)

Saving weights ...
Store weights of layer 00Conv2D_6x24x24 to file /home/wolin/snn_toolbox/examples/models/lenet5/keras/98.96_brian2/brian2-model/00Conv2D_6x24x24.npz
Store weights of layer 01MaxPooling2D_6x12x12 to file /home/wolin/snn_toolbox/examples/models/lenet5/keras/98.96_brian2/brian2-model/01MaxPooling2D_6x12x12.npz
Store weights of layer 02Conv2D_16x8x8 to file /home/wolin/snn_toolbox/examples/models/lenet5/keras/98.96_brian2/brian2-model/02Conv2D_16x8x8.npz
Store weights of layer 03MaxPooling2D_16x4x4 to file /home/wolin/snn_toolbox/examples/models/lenet5/keras/98.96_brian2/brian2-model/03MaxPooling2D_16x4x4.npz
Store weights of layer 04Conv2D_120x4x4 to file /home/wolin/snn_toolbox/examples/models/lenet5/keras/98.96_brian2/brian2-model/04Conv2D_120x4x4.npz
Store weights of layer 06Dense_84 to file /home/wolin/snn_toolbox/examples/models/lenet5/keras/98.96_brian2/brian2-model/06Dense_84.npz
Store weights of layer 07Dense_10 to file /home/wolin/snn_toolbox/examples/models/len

In [31]:
config

In [32]:
results = run(spiking_model, **testset)


Starting new simulation...

Current accuracy of batch:
Starting simulation at t=0. s for a duration of 100. ms
0.009000000000000001 (9%) simulated in < 1s, estimated < 1s remaining.
0.021 (21%) simulated in < 1s, estimated < 1s remaining.
0.033 (33%) simulated in < 1s, estimated < 1s remaining.
0.047 (47%) simulated in < 1s, estimated < 1s remaining.
0.06 (60%) simulated in < 1s, estimated < 1s remaining.
0.07100000000000001 (71%) simulated in < 1s, estimated < 1s remaining.
0.084 (84%) simulated in < 1s, estimated < 1s remaining.
0.098 (98%) simulated in < 1s, estimated < 1s remaining.
0.1 (100%) simulated in < 1s

Batch 1 of 10 completed (10.0%)
Moving accuracy of SNN (top-1, top-1): 100.00%, 100.00%.
Moving accuracy of ANN (top-1, top-1): 100.00%, 100.00%.

Calculating activations...

Average spike rate: 0.019532031436056203 spikes per simulation time step.
Saving plots of one sample to /home/wolin/snn_toolbox/examples/models/lenet5/keras/log/gui/test...

Plotting layer 00Conv2D_6x

0.013999999999999999 (13%) simulated in < 1s, estimated < 1s remaining.
0.028999999999999998 (28%) simulated in < 1s, estimated < 1s remaining.
0.045 (44%) simulated in < 1s, estimated < 1s remaining.
0.061 (61%) simulated in < 1s, estimated < 1s remaining.
0.077 (76%) simulated in < 1s, estimated < 1s remaining.
0.093 (93%) simulated in < 1s, estimated < 1s remaining.
0.1 (100%) simulated in < 1s

Batch 8 of 10 completed (80.0%)
Moving accuracy of SNN (top-1, top-1): 100.00%, 100.00%.
Moving accuracy of ANN (top-1, top-1): 100.00%, 100.00%.

Calculating activations...

Average spike rate: 0.016153846153846154 spikes per simulation time step.
Saving plots of one sample to /home/wolin/snn_toolbox/examples/models/lenet5/keras/log/gui/test...

Plotting layer 00Conv2D_6x24x24
Plotting layer 01MaxPooling2D_6x12x12
Plotting layer 02Conv2D_16x8x8
Plotting layer 03MaxPooling2D_16x4x4
Plotting layer 04Conv2D_120x4x4
Plotting layer 06Dense_84
Plotting layer 07Dense_10
Plotting batch run statisti

WARNING    /home/wolin/snn_toolbox/snntoolbox/simulation/utils.py:745: RuntimeWarning: invalid value encountered in true_divide
  avg_acc = np.mean(np.true_divide(match, count))
 [py.warnings]


Simulation finished.


Total accuracy: 100.00% on 10 test samples.


Accuracy averaged over classes: nan%
